## Project Euler 
#### Questions 31-40

Continuing the quest to solve every Euler question!

---
**Problem 31:** Coin sums

In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation: 1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).

How many different ways can £2 be made using any number of coins?

In [76]:
#Make a list of all coins:
coins = [200,100,50,20,10,5,2,1]

In [78]:
#This is a classic recursion problem!
def ways_possible_change(input_coin_list, amount_to_change):
    '''Input a set of coins with set values and any amount of money, 
    output the number of possible combinations of coins to make that amount.'''
    
    input_coin_list.sort()
    coin_list = input_coin_list[::-1]
    coin_list = [coin for coin in coin_list if coin <= amount_to_change]
    
    #Fully divided ammount
    if amount_to_change == 0:
        return 1
    
    #One coin left
    elif len(coin_list) == 1:
        if amount_to_change % coin_list[0] == 0:
            return 1
        else:
            return 0
    
    #Coin equals amount to change
    elif coin_list[0] == amount_to_change:
        return ways_possible_change(coin_list[1:], amount_to_change)+1
    
    else:
        partitions = amount_to_change // coin_list[0]
        change_left_over = [amount_to_change - coin_list[0]*(i) for i in range(int(partitions)+1)]
        return sum([ways_possible_change(coin_list[1:], amount) for amount in change_left_over])

In [77]:
ways_possible_change(coins, 200)

73682

In [79]:
#For fun, ways to change a dollar:
ways_possible_change([25,10,5,1], 100)

242

---
**Problem 32:** Pandigital Products
We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.

The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital.

Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.

HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.